In [ ]:
# Parcial: Curso Análisis Predictivo de Series Temporales

## Posgrado de Analítica de Big Data

## Universidad ORT Uruguay

### Curso 2020

**Fecha límite de entrega:** 17/7/2020, 21:00hs. por el [sitio de gestion](http://gestion.ort.edu.uy).

In [ ]:
## Ejercicio

El ejercicio consiste en analizar diferentes modelos para la serie de muertes (cada 10.000 habitantes) relacionadas con influenza y neumonia en EEUU, disponible en la biblioteca `astsa`. La serie, denominada `flu`, comienza en 1968 y toma valores mensuales hasta 1978 inclusive. 

Se propone:

1. Realizar un análisis exploratorio de la serie, determinando tendencias y posibles estacionalidades. Analizar también la autocorrelación de la serie.

2. Realizar un ajuste lineal basado en tendencia y factores estacionales (en principio uno por mes).

3. Realizar un ajuste lineal basado en tendencia y frecuencias. Justificar la elección de las frecuencias incluidas en el ajuste.

4. Comparar los ajustes anteriores desde el punto de vista del error cometido. ¿Con cuál se quedaría si debiera elegir? Justifique.

5. A los residuos de ambos ajustes anteriores, analizar si es posible modelarlos mediante un proceso ARMA(p,q) adecuado. Justifique la elección del orden en cada caso y analice los residuos obtenidos (no importa si no son Gaussianos).

6. Discutir cuál de los caminos seguidos logra mejor ajuste luego de ajustar los residuos.

7. Realizar una predicción completa para los años 1979 y 1980 de la serie a partir del ajuste que crea conveniente.

8. Se dispone además de una serie de temperaturas medias mensuales en EEUU en el archivo `temperaturas.csv`. Analizar:
 * Si hay correlación cruzada entre la serie de muertes por gripe y neumonia y la temperatura observada en cada mes.
 
 * Si esta correlación se mantiene una vez que se saca el efecto estacional a la temperatura.
 
 
**Nota:** Para el análisis pueden utilizarse todas las bibliotecas vistas en el curso así como otras que entiendan convenientes. Se recomienda en particular `forecast` para algunos ajustes y predicciones. 

In [ ]:
install.packages("astsa")
install.packages("forecast")
library(lmtest)
library(forecast)
library(astsa)
tsplot(flu)
options(repr.plot.width=10, repr.plot.height=6) #ajusta tamaño de graficas

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


In [ ]:
flu

### $Parte 1:$ 
Realizar un análisis exploratorio de la serie, determinando tendencias y posibles estacionalidades. Analizar también la autocorrelación de la serie.

In [ ]:
plot(decompose(flu)) #Analizamos la descomposicion de la serie

In [ ]:
#Le realizamos el periodigrama a los residuos del Fit_0 para sacarle la tendencia.

Fit_0 = lm(flu~t)

n<-length(residuals(Fit_0))
I = abs(fft(residuals(Fit_0)))^2/n 
P = (4/n)*I[1:(n/2)]
f = 12 * (0:(n/2-1))/n

plot(f, P, type="l", col = 8, xlab="Frequency", ylab="Scaled Periodogram")
tsplot(f,P, xlim=c(0,10))

𝑂𝑏𝑠𝑒𝑟𝑣𝑎𝑐𝑖ó𝑛:  Por medio del periodograma podemos observar que existe un componente periodico, según la gráfica anterior se observa claramente un peirodo que se repite de forma anual, lo que indica que hay un ciclo periodico cada año. También se puede observar que existe pero mucho mas leve tambien internamente dos ciclos anuales.
En el decompose de mas arriba se puede observar una leve tendencia decreciente de la serie.

### $Parte2:$
Realizar un ajuste lineal basado en tendencia y factores estacionales (en principio uno por mes).


In [ ]:
season = factor(cycle(flu)) 
t = time(flu)-1968

Fit_1 = lm(flu ~ 0 + t + season )
summary(Fit_1)

In [ ]:
acf2(residuals(Fit_1), , col = 8)

In [ ]:
Ajuste_1 = ts(fitted(Fit_1),start=1968,freq=12)

tsplot(flu)
lines(Ajuste_1,col=12)

### $Parte 3:$
Realizar un ajuste lineal basado en tendencia y frecuencias. Justificar la elección de las frecuencias incluidas en el ajuste. 

In [ ]:
#Viendo el periodroigrma de la parte 1 utilizamos la frecuencia para un modelo seno y coseno con frecuencia 1 y 2. 
#También comparamos con el modelo de frecuencia 1 solo, y comparamos los modelos segun los residuos.

Fit_2_prueba<-lm(flu~t+sin(2*pi*t)+cos(2*pi*t))

Fit_2<-lm(flu~t+sin(2*pi*t)+cos(2*pi*t)+sin(4*pi*t)+cos(4*pi*t))


SSE_2_prueba = sum(residuals(Fit_2_prueba)^2) #Modelo lineal con frecuencia 1
SSE_2 = sum(residuals(Fit_2)^2) #Modelo lineal con frecuencia 1 y 2


R2 = (SSE_2_prueba-SSE_1)/SSE_2_prueba

#muestro los valores

SSE_2_prueba
SSE_2
R2


# El mejor modelo es que tiene frecuencia 1 y 2 dado que tiene menor SSE 

In [ ]:
summary(Fit_2)

acf(residuals(Fit_2), , col = 8)
pacf(residuals(Fit_2),  col = 8)

In [ ]:
Ajuste_2 = ts(fitted(Fit_2),start=1968,freq=12)

tsplot(flu)
lines(Ajuste_2, col=2)

### $Parte4$:
Comparar los ajustes anteriores desde el punto de vista del error cometido. ¿Con cuál se quedaría si debiera elegir? Justifique.



In [ ]:
## Para comparar los ajustes de los modelos y estudiar el error cometido, recurrimos al estimador de suma de cuadrado de los residuos.

SSE = sum((flu-mean(flu))^2) #Este se hace para ver como mejora sacandole la media
SSE_1 = sum(residuals(Fit_1)^2) #Modelo lineal 1
SSE_2 = sum(residuals(Fit_2)^2) #Modelo lineal 2



R2 = (SSE_2-SSE_1)/SSE_2

#muestro los valores
SSE
SSE_1
SSE_2
R2 #El R^2 es una medida de correlación de nuestras variables, o bien cuánto mejora el ajuste en términos relativos respecto a la media.

### Lo que se puede observar que el modelo 1 (Fit_1) en terminos relativos tiene un mejor ajuste del 10% aprox en comparacion con el modelo 2 (Fit_2)

### $Parte 5:$ 
A los residuos de ambos ajustes anteriores, analizar si es posible modelarlos mediante un proceso ARMA(p,q) adecuado. Justifique la elección del orden en cada caso y analice los residuos obtenidos (no importa si no son Gaussianos).



In [ ]:
# Modelo lineal 1
y = residuals(Fit_1)
head(y)
acf2(y)


In [ ]:
#Según los valors del acf y pacf, le ajustamos un modelo MA(1) y un AR(2)
Fit_1.2 = Arima(y , c(2,0,1),include.mean = FALSE) 
summary(Fit_1.2)

In [ ]:
acf2(residuals(Fit_1.2))

In [ ]:
# Modelo lineal 2
j = residuals(Fit_2)
head(j)
acf2(j)

In [ ]:
#Según los valors del acf y pacf, le ajustamos un modelo MA(1) y un AR(2)
Fit_2.2 = Arima(j, c(2,0,1),include.mean = FALSE) 
summary(Fit_2.2)

In [ ]:
acf2(residuals(Fit_2.2))

### $Parte 6:$

Discutir cuál de los caminos seguidos logra mejor ajuste luego de ajustar los residuos.



In [ ]:
## Analizamos los residuos de los modelos

tsplot(residuals(Fit_1.2))
checkresiduals(Fit_1.2)

# El p-valor nos queda mayor que 0.05 por lo que se puede decir que los residuos son ruido blanco

In [ ]:
# Ajuste de los residuos del modelo 2

tsplot(residuals(Fit_2.2))
checkresiduals(Fit_2.2)

#En este caso el p-valor quedo por encima del 0,05 y también los residuos son ruido blanco.

In [ ]:
AIC(Fit_1.2)
AIC(Fit_2.2)
BIC(Fit_1.2)
BIC(Fit_2.2)

### Como los valores de AIC (criterio de máxima verosimilitud) y BIC (el estadístico BIC aumenta la penalidad en modelos con muchas variables y dicho estadistico aumenta la penalidad en modelos con muchas variables ) quedaron mas bajos en el modelo 2 ajustados por los residuos un AR(3) y un MA(2). Esto se contrasta con la primera analisis comparativo donde el modelo 1 era mejor pero podiamos caer en overfiting dado que el modelo tenia muchos parámetros.

### $Parte 7:$

Realizar una predicción completa para los años 1979 y 1980 de la serie a partir del ajuste que crea conveniente.



In [ ]:
Pred_1= forecast(Fit_2.2,h=24) 
tsplot(Pred_1,col="blue", lwd=2, main = "Predicción fit" ,xlab = "Tiempo", ylab = "")

In [ ]:
## Construimos los nuevos datos a evaluar el modelo

#Agregamos tiempos para 4 años en el futuro. Notar el -1970 para que quede igual que la que ajustamos.
new_t = seq(1979,1980.917,by=0.083) - 1970
#Lo convierto en serie temporal
new_t = ts(new_t,start=1979,freq=12)

#Predigo sobre los nuevos datos.
new_data = data.frame(t=new_t)
prediccion_1= predict(Fit_2, new_data);

#Convierto el resultado en time series
prediccion_final = ts(prediccion_1,start=1979,freq=12)
prediccion_final


In [ ]:
tsplot(flu,xlim=c(1970,1982),ylim=c(0,0.7))
lines(Ajuste_2,col=2)
lines(prediccion_final,col=3)

In [ ]:
# Armamos la predicion total con los dos modelos (lm y ARMA)

pred1_hat= ts(Pred_1$fitted,start = 1978, frequency = 12)
print(head(pred1_hat))

pred2_hat = ts(prediccion_final,start = 1978, frequency = 12)
print(head(pred2_hat))

x_hat = pred1_hat + pred2_hat

In [ ]:
tsplot(flu,xlim=c(1968,1980),ylim=c(0,1))
lines(x_hat,col=2)

### $Parte 8$: Se dispone además de una serie de temperaturas medias mensuales en EEUU en el archivo `temperaturas.csv`. Analizar:
 * Si hay correlación cruzada entre la serie de muertes por gripe y neumonia y la temperatura observada en cada mes.
 * Si esta correlación se mantiene una vez que se saca el efecto estacional a ambas series. 

In [ ]:
temp = ts(read.csv("temperaturas.csv")[,"Value"],start=1968,freq=12)
tsplot(temp)

### Para estudiar la correlacion cruzada entre la serie temp y flu lo que vamos vamos a realizar es lo siguientes:
1- Estudiar la correlacion cruzada con la funcion ccf, cor para el coeficiente de correlacion y estudiar las graficas que se generar con la función lag2.plot con sus respectivos lags.
2- Realizar un modelo basico linear con la serie temp para estudiar los residuos y poder compararlos sin el efecto estacional.
3- Estudiar nuevamente la correlacion cruzada y las graficas para ver como es la correlacion entre las series luego de sacarle el efecto estacional.

In [ ]:
## Paso 1

ccf(flu,temp)
ccf_Values_1 = ccf(flu,temp)
ccf_Values_1

In [ ]:
lag2.plot (flu, temp, 10)

In [ ]:
cor(flu,temp)

In [ ]:
## Paso 2

season = factor(cycle(temp)) 
t = time(temp)-1968

Fit_temp = lm(temp ~ 0 + t + season )
summary(Fit_temp)

In [ ]:
## Paso 3

resid_Fit_temp = ts(residuals(Fit_temp), start = 1968, freq = 12)
resid_Fit_1 = ts(residuals(Fit_1), start = 1968, freq = 12)
tsplot(resid_Fit_temp)

In [ ]:
ccf(resid_Fit_temp,resid_Fit_1)
ccf_Values_2 = ccf(resid_Fit_temp,resid_Fit_1)
ccf_Values_2



In [ ]:
lag2.plot (resid_Fit_temp,resid_Fit_1, 10)

In [ ]:
cor(resid_Fit_temp,resid_Fit_1)

### $ Conclusiones: $

Lo que se puede ver es que en el paso 1 donde se estudian las series en su forma normal, existe cierta correlación entre ambas y tanto en el ccf como en las gráficas se pueden especificar con los determinados lags. También si vemos el coeficiente de correlación nos da -0,67.
Por otro lado al sacarle los factores estacionales a las series y realizamos los mismos estudios, se puede observar como la correlación se elimina casi completamente, con un cor de 0.037. Lo mismo se puede ver en los coeficientes de correlación en las gráficas y en el ccf.